In [1]:
"""dimension annotation
b: batch
t: token position
d: d_model

t64 means it's still a t dimension with size of 64
"""

from functools import partial

import numpy as np
import torch

import transformer_lens.utils as utils
from transformer_lens import HookedTransformer
from tqdm import tqdm

from openwebtext import load_owt, sample
from pretrained_sae import load_sae

torch.set_grad_enabled(False)

layer_index = 6
location = "resid_post_mlp"
device = utils.get_device()

ds = load_owt()
gpt2 = HookedTransformer.from_pretrained("gpt2", center_writing_weights=False)

sae_32k = load_sae(32, location, layer_index, device)
sae_128k = load_sae(128, location, layer_index, device)

Loading dataset from disk:   0%|          | 0/152 [00:00<?, ?it/s]

Loaded 8,013,769 sample texts from data/owt_tokenized


/home/lukaemon/miniconda3/envs/topk_sae/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loaded pretrained model gpt2 into HookedTransformer
Loaded pretrained SAE data/sae/v5_32k_location_resid_post_mlp_layer_6.pt
Loaded pretrained SAE data/sae/v5_128k_location_resid_post_mlp_layer_6.pt


In [2]:
def hook_fn_reconstruct_act(act_btd, hook, sae):
    latent, info = sae.encode(act_btd)
    recon_act_btd = sae.decode(latent, info)

    return recon_act_btd

In [3]:
# dry run
# batch = sample(ds, 8)
# loss = gpt2(batch, return_type='loss')
# loss_sae = gpt2.run_with_hooks(
#     batch,
#     return_type='loss',
#     fwd_hooks=[(
#         utils.get_act_name('resid_post', layer_index),
#         partial(hook_fn_reconstruct_act, sae=sae_32k)
#     )]
# )

# print(f"loss {loss.item():.3f}, loss_sae {loss_sae.item():.3f}")

In [4]:
n_batch = 256
batch_size = 128

loss = []
loss_sae32k = []
loss_sae128k = []

for _ in tqdm(range(n_batch), unit='batch'):
    batch_bt64 = sample(ds, batch_size)

    l = gpt2(batch_bt64, return_type='loss')
    l32 = gpt2.run_with_hooks(
        batch_bt64,
        return_type='loss',
        fwd_hooks=[(
            utils.get_act_name('resid_post', layer_index),
            partial(hook_fn_reconstruct_act, sae=sae_32k)
        )]
    )
    l128 = gpt2.run_with_hooks(
        batch_bt64,
        return_type='loss',
        fwd_hooks=[(
            utils.get_act_name('resid_post', layer_index),
            partial(hook_fn_reconstruct_act, sae=sae_128k)
        )]
    )

    loss.append(l.item())
    loss_sae32k.append(l32.item())
    loss_sae128k.append(l128.item())


100%|██████████| 256/256 [03:40<00:00,  1.16batch/s]


In [5]:
print(f"loss {np.mean(loss): .3f}")
print(f"sae 32k loss {np.mean(loss_sae32k): .3f}")
print(f"sae 128k loss {np.mean(loss_sae128k): .3f}")

loss  3.729
sae 32k loss  3.831
sae 128k loss  3.789


In [9]:
print(f'delta loss, sae 32k {np.mean(loss_sae32k) - np.mean(loss):.5f}')
print(f'delta loss, sae 128k {np.mean(loss_sae128k) - np.mean(loss):.5f}')

delta loss, sae 32k 0.10235
delta loss, sae 128k 0.06063
